In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.impute import SimpleImputer

import os
from pathlib import Path

In [ ]:
current_path = Path().resolve()
while current_path.name != "ADS_Final":
    if current_path.parent == current_path:
        raise FileNotFoundError("❌ Không tìm thấy thư mục gốc 'ADS_Final'")
    current_path = current_path.parent

os.chdir(current_path)

In [ ]:
students = pd.read_csv("Data/diemthi.csv")

programs = pd.read_csv("Data/Final/diem_chuan_full.csv")

In [ ]:
subjects = ['Toán', 'Văn', 'Sử', 'Địa', 'Lí', 'Hóa', 'Sinh', 'Ngoại ngữ', 'GDCD']
students[subjects] = students[subjects].apply(pd.to_numeric, errors='coerce')

In [ ]:
students.head()

In [ ]:
students_df = students.rename(columns={'Năm thi':'Year'})
programs_df = programs.rename(columns={'Năm':'Year'})

In [ ]:
def parse_scale(loai_diem):
    if 'Thang 40' in loai_diem:
        return 40
    elif 'Thang 10' in loai_diem:
        return 10
    else:
        # default fallback
        return 30

In [ ]:
programs_df['Scale'] = programs_df['Loại điểm'].apply(parse_scale)

In [ ]:
block_to_subjects = {
    "A00": ["Toán", "Lí", "Hóa"],
    "A01": ["Toán", "Lí", "Ngoại ngữ"],
    "B00": ["Toán", "Hóa", "Sinh"],
    "C00": ["Văn", "Sử", "Địa"],
    "C19": ["Văn", "Toán", "Khoa học tự nhiên"],
    "C20": ["Toán", "Sinh", "Khoa học xã hội"],
    "D01": ["Toán", "Văn", "Ngoại ngữ"],

}


In [ ]:
programs_df['BlockCode'] = programs_df['Tổ hợp']

In [ ]:
programs_df= programs_df[programs_df['BlockCode'].isin(block_to_subjects.keys())]

In [ ]:
def parse_weights(row):
    """
    Returns a dict of { subject_name: weight } for that block.
    Defaults to weight=1 for each of the 3 subjects,
    but if Ghi chú contains "Anh hệ số 2" or similar, we bump English to weight=2.
    """
    block = row['BlockCode']
    needed = block_to_subjects[block]
    weights = { subj: 1 for subj in needed }

    note = row.get('Ghi chú', "")
    if not isinstance(note, str):
        note = ""
    if "CLC" in note:

        weights['Ngoại ngữ'] = 2

    return weights


programs_df['Weights'] = programs_df.apply(parse_weights, axis=1)

In [ ]:
programs_df

In [ ]:
def compute_weighted_block_score(row):
    block = row['BlockCode']
    weights = row['Weights']         # e.g. {'Toán':1,'Văn':1,'Ngoại ngữ':2}
    score = 0.0
    for subj, w in weights.items():
        val = row.get(subj, 0)
        if pd.isna(val):
            val = 0
        score += val * w
    return score

programs_df['BlockScore'] = programs_df.apply(compute_weighted_block_score, axis=1)

In [ ]:
data= pd.read_csv("Data/Model Data/dan_nhan_2021.csv")

In [ ]:
data.head()

In [ ]:
def sample_n(x, n=4, seed=42):
    return x.sample(n=min(len(x), n), random_state=seed)

reduced_df = (
    data
    .groupby('SBD', group_keys=False)
    .apply(lambda grp: sample_n(grp, n=4))
)

In [ ]:
reduced_df.to_csv("model/reduced_df.csv", index=False, encoding='utf-8-sig')

In [ ]:
drive_path = "model/reduced_df.csv"
reduced_df=pd.read_csv(drive_path)

In [ ]:
reduced_df


,SBD,Sở GD,Toán,Văn,Sử,Địa,Lí,Hóa,Sinh,Ngoại ngữ,GDCD,Năm thi,Tên ngành trúng tuyển,Tên trường trúng tuyển
0,2000001,Sở GD&ĐT Thành Phố Hồ Chí Minh,8.2,7.00,NaN,NaN,8.0,8.5,7.75,8.2,NaN,2021,Bất động sản,Đại Học Dân Lập Văn Lang
1,2000001,Sở GD&ĐT Thành Phố Hồ Chí Minh,8.2,7.00,NaN,NaN,8.0,8.5,7.75,8.2,NaN,2021,Khoa học & Quản lý môi trường,Đại Học Nông Lâm – Đại Học Thái Nguyên
2,2000001,Sở GD&ĐT Thành Phố Hồ Chí Minh,8.2,7.00,NaN,NaN,8.0,8.5,7.75,8.2,NaN,2021,Khoa học máy tính,Đại Học Dân Lập Duy Tân
3,2000001,Sở GD&ĐT Thành Phố Hồ Chí Minh,8.2,7.00,NaN,NaN,8.0,8.5,7.75,8.2,NaN,2021,Quản trị nhà hàng khách sạn,Đại Học Tôn Đức Thắng
4,2000002,Sở GD&ĐT Thành Phố Hồ Chí Minh,8.8,7.00,7.75,8.0,NaN,NaN,NaN,9.8,9.0,2021,Marketing,Đại Học Tài Chính Marketing
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39968,2010428,Sở GD&ĐT Thành Phố Hồ Chí Minh,7.6,6.25,3.50,6.5,NaN,NaN,NaN,8.0,8.0,2021,Giáo dục Tiểu học,Đại Học Sư Phạm – Đại Học Huế
39969,2010429,Sở GD&ĐT Thành Phố Hồ Chí Minh,8.6,6.50,NaN,NaN,8.0,5.5,7.00,7.6,NaN,2021,Tài chính - Ngân hàng,Đại Học Trà Vinh
39970,2010429,Sở GD&ĐT Thành Phố Hồ Chí Minh,8.6,6.50,NaN,NaN,8.0,5.5,7.00,7.6,NaN,2021,Quản trị kinh doanh,Đại Học Hòa Bình
39971,2010429,Sở GD&ĐT Thành Phố Hồ Chí Minh,8.6,6.50,NaN,NaN,8.0,5.5,7.00,7.6,NaN,2021,Công nghệ kỹ thuật năng lượng,Đại Học Điện Lực


In [ ]:
grouped = (
    reduced_df
    .groupby('SBD', as_index=False)
    .agg({
        'Toán': 'first',
        'Văn':  'first',
        'Sử':   'first',
        'Địa':  'first',
        'Lí':   'first',
        'Hóa':  'first',
        'Sinh': 'first',
        'Ngoại ngữ': 'first',
        'GDCD': 'first',
        # collect all unique majors into a list
        'Tên ngành trúng tuyển': lambda x: list(pd.unique(x))
    })
)

In [ ]:
grouped

,SBD,Toán,Văn,Sử,Địa,Lí,Hóa,Sinh,Ngoại ngữ,GDCD,Tên ngành trúng tuyển
0,2000001,8.2,7.00,NaN,NaN,8.00,8.50,7.75,8.2,NaN,"[Bất động sản, Khoa học & Quản lý môi trường, ..."
1,2000002,8.8,7.00,7.75,8.00,NaN,NaN,NaN,9.8,9.00,"[Marketing, Phân tích dữ liệu kinh doanh, Khoa..."
2,2000003,7.8,6.50,7.75,7.25,NaN,NaN,NaN,9.2,9.25,"[Kế toán, Luật, Tài chính - Ngân hàng, Kinh tế]"
3,2000004,6.6,7.00,6.00,7.50,NaN,NaN,NaN,9.6,9.25,"[Kinh tế, Thương mại điện tử, Công nghệ Kỹ thu..."
4,2000005,8.0,6.75,NaN,NaN,5.75,8.00,7.25,8.6,NaN,"[Kỹ thuật trắc địa - bản đồ, Sư phạm Địa lý, C..."
...,...,...,...,...,...,...,...,...,...,...,...
9995,2010425,7.2,6.75,NaN,NaN,3.00,2.00,3.50,9.0,NaN,"[Kỹ thuật máy tính, Ngôn ngữ Anh, Khoa học môi..."
9996,2010426,8.8,6.50,NaN,NaN,7.25,7.75,5.25,NaN,NaN,"[Kỹ thuật phần mềm, Công nghệ kỹ thuật điện, đ..."
9997,2010427,8.2,6.75,NaN,NaN,7.25,4.25,4.75,9.0,NaN,"[Quản trị khách sạn, Quản trị dịch vụ du lịch ..."
9998,2010428,7.6,6.25,3.50,6.50,NaN,NaN,NaN,8.0,8.00,"[Công nghệ kỹ thuật vật liệu xây dựng, Phát tr..."


In [ ]:
def clean_majors(x):
    # If it’s not a list, wrap it into one
    if not isinstance(x, list):
        x = [x]
    # Filter out any nulls, convert everything to str
    cleaned = [str(label).strip() for label in x if pd.notna(label)]
    return cleaned

In [ ]:
grouped['majors_list'] = grouped['Tên ngành trúng tuyển'].apply(clean_majors)

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer(sparse_output=True)
# learns the 345 distinct majors
Y_multi = mlb.fit_transform(grouped['majors_list'])
# Y_multi.shape == (n_students, 345)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    accuracy_score,
    hamming_loss,
    precision_recall_fscore_support,
    roc_auc_score,
    classification_report
)

In [ ]:
feature_cols = ['Toán','Văn','Sử','Địa','Lí','Hóa','Sinh','Ngoại ngữ','GDCD']
X = grouped[feature_cols].fillna(0).values  # shape (40k, 9)

In [ ]:
Y = Y_multi.toarray()
X_train, X_val, Y_train, Y_val = train_test_split(
    X, Y, test_size=0.2, random_state=42
)

# Base classifier
base = LogisticRegression(
    penalty='l2',
    C=1.0,
    solver='saga',
    max_iter=1000,
    n_jobs=-1
)

# Wrap for multi-label
clf = OneVsRestClassifier(LogisticRegression(solver='saga', max_iter=1000), n_jobs=-1)
clf.fit(X_train, Y_train)

# Predict probabilities for each label
Y_pred_proba = np.array([est.predict_proba(X_val)[:,1] for est in clf.estimators_]).T


In [ ]:
import joblib

In [ ]:
joblib.dump(mlb, 'model/mlb_majors.pkl')
joblib.dump(clf, 'model/clf_multilabel.pkl')
joblib.dump(feature_cols,'model/score.pkl')

In [ ]:
# from sklearn.neighbors import NearestNeighbors
# from collections import Counter
# def recall_at_k_for_knn(k_neighbors, k_recommend):
#     knn = NearestNeighbors(n_neighbors=k_neighbors, metric='euclidean', n_jobs=-1)
#     knn.fit(X_train)
#     dists, idxs = knn.kneighbors(X_val)
#     count = 0
#     for true, neigh in zip(Y_val, idxs):
#         recs = Counter(sum((admissions[i] for i in neigh), [])).most_common(k_recommend)
#         rec_majors = [m for m,_ in recs]
#         if true in rec_majors:
#             count += 1
#     return count / len(Y_val_lists)

# for k_n in [20,50,100]:
#     for k_r in [3,5,10]:
#         print(f"KNN={k_n}, Recall@{k_r} = {recall_at_k_for_knn(k_n,k_r):.4f}")